# Wavelet Divergence & Quality Vector Analysis
## Project SHIELD - Sensor Health Monitoring Pipeline

This notebook demonstrates:
1. Loading engine team sensor data (L3GD20 gyro, LIS3DH accel) and PAMAP2 IMU data
2. Enhanced MODWT feature extraction (energy, variance, entropy, RMS, kurtosis per sub-band)
3. KL divergence (belief divergence) analysis across time windows
4. Stable vs sensitive feature identification
5. Quality vector construction for sensor health prediction

**Architecture**: Raw Signal → MODWT (sym4, 5 levels) → Per-level stats + Signal Quality Features → Divergence Analysis → Quality Vector

In [ ]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

sys.path.insert(0, str(Path('../src')))
from physics_based_classification.feature_extractor import FeatureExtractor
from physics_based_classification.wavelet_analyses import (
    extract_wavelet_features, modwt_fast
)
from physics_based_classification.signal_quality import extract_signal_quality_features
from physics_based_classification.divergence_analysis import (
    build_reference_distributions, compute_window_divergence,
    compute_cross_sensor_divergence
)
from physics_based_classification.quality_vector import (
    assess_feature_stability, assess_cross_condition_stability,
    select_quality_features, build_quality_vector,
    DEFAULT_MANDATORY_FEATURES
)

%matplotlib inline
plt.rcParams['figure.dpi'] = 120
plt.rcParams['figure.facecolor'] = 'white'

## 1. Load Engine Team Data

In [ ]:
data_dir = Path('../datasets/Engine Team Data')

# Load calibrated gyro data (~4.3 hours, ~48 Hz)
gyro_df = pd.read_csv(data_dir / 'gyro_data_20260201-195148 (1).csv')
gyro_df = gyro_df.sort_values('Time_ms').reset_index(drop=True)
gyro_fs = 48.0

# Use calibrated values
gyro_cal_x = gyro_df['CalX'].values.astype(np.float64)
gyro_cal_y = gyro_df['CalY'].values.astype(np.float64)
gyro_cal_z = gyro_df['CalZ'].values.astype(np.float64)

# Also load raw for comparison
gyro_raw_x = gyro_df['RawX'].values.astype(np.float64)

print(f'Gyro: {len(gyro_df):,} samples, {len(gyro_df)/gyro_fs/3600:.1f} hours')
print(f'  CalX range: [{gyro_cal_x.min():.1f}, {gyro_cal_x.max():.1f}]')
print(f'  Time span: {(gyro_df["Time_ms"].max() - gyro_df["Time_ms"].min())/1000/60:.1f} min')

In [ ]:
# Load accel data (~4 hours, ~83 Hz) - timestamps NOT sorted
accel_df = pd.read_csv(data_dir / 'lis3dh_accel_20260209-202619.csv')
accel_df = accel_df.sort_values('Time_ms').reset_index(drop=True)
accel_fs = 83.0

accel_x = accel_df['Accel_X'].values.astype(np.float64)
accel_y = accel_df['Accel_Y'].values.astype(np.float64)
accel_z = accel_df['Accel_Z'].values.astype(np.float64)

print(f'Accel: {len(accel_df):,} samples, {len(accel_df)/accel_fs/3600:.1f} hours')
print(f'  X range: [{accel_x.min():.2f}, {accel_x.max():.2f}] m/s²')
print(f'  Z mean: {accel_z.mean():.2f} m/s² (expect ~9.81 gravity)')

In [ ]:
# Load SCD41 environmental data for reference
env_df = pd.read_csv(data_dir / 'scd41_data_20260208-201210.csv')
env_df = env_df.sort_values('Time_ms').reset_index(drop=True)
print(f'Environment: {len(env_df)} samples, {len(env_df)/0.2/3600:.1f} hours')
print(f'  CO2: {env_df["CO2_ppm"].min()}-{env_df["CO2_ppm"].max()} ppm')
print(f'  Temp: {env_df["Temp_C"].min():.1f}-{env_df["Temp_C"].max():.1f} °C')

## 2. Load PAMAP2 IMU Reference Data

In [ ]:
from SensorDataLoader import SensorDataLoader

loader = SensorDataLoader(seed=42)
pamap2_path = Path('../datasets/PAMAP2_Dataset/Protocol')

if pamap2_path.exists():
    pamap2 = loader.load_pamap2(str(pamap2_path / 'subject101.dat'))
    # Get a gyro and accel signal from hand sensor
    pamap2_gyro = pamap2.get('hand_gyro', None)
    pamap2_accel = pamap2.get('hand_accel_16g', None)
    pamap2_fs = 100.0
    
    if pamap2_gyro is not None:
        pamap2_gyro_x = pamap2_gyro[:, 0]
        print(f'PAMAP2 gyro: {len(pamap2_gyro_x):,} samples @ {pamap2_fs} Hz')
    if pamap2_accel is not None:
        pamap2_accel_x = pamap2_accel[:, 0]
        print(f'PAMAP2 accel: {len(pamap2_accel_x):,} samples @ {pamap2_fs} Hz')
else:
    print('PAMAP2 not found - skipping IMU reference comparison')
    pamap2_gyro_x = None
    pamap2_accel_x = None
    pamap2_fs = None

## 3. MODWT Decomposition Visualization

In [ ]:
# MODWT decomposition of a segment of engine gyro
seg_len = int(10 * gyro_fs)  # 10 seconds
gyro_seg = gyro_cal_x[:seg_len]

decomp = modwt_fast(gyro_seg, level=5)
n_levels = decomp['levels']
t = np.arange(len(gyro_seg)) / gyro_fs

fig, axes = plt.subplots(n_levels + 2, 1, figsize=(14, 12), sharex=True)
axes[0].plot(t, gyro_seg, linewidth=0.5, color='black')
axes[0].set_ylabel('Original')
axes[0].set_title('MODWT Decomposition - Engine Gyro CalX (10s segment)')

for j in range(1, n_levels + 1):
    axes[j].plot(t, decomp[f'D{j}'], linewidth=0.5, color=f'C{j-1}')
    freq_hi = gyro_fs / (2**j)
    freq_lo = gyro_fs / (2**(j+1))
    axes[j].set_ylabel(f'D{j}\n({freq_lo:.1f}-{freq_hi:.1f}Hz)')

axes[-1].plot(t, decomp[f'A{n_levels}'], linewidth=0.5, color='gray')
axes[-1].set_ylabel(f'A{n_levels}\n(baseline)')
axes[-1].set_xlabel('Time (s)')

plt.tight_layout()
os.makedirs('figures', exist_ok=True)
plt.savefig('figures/modwt_engine_gyro_decomposition.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Same for accel
seg_len_a = int(10 * accel_fs)
accel_seg = accel_x[:seg_len_a]

decomp_a = modwt_fast(accel_seg, level=5)
t_a = np.arange(len(accel_seg)) / accel_fs

fig, axes = plt.subplots(n_levels + 2, 1, figsize=(14, 12), sharex=True)
axes[0].plot(t_a, accel_seg, linewidth=0.5, color='black')
axes[0].set_ylabel('Original')
axes[0].set_title('MODWT Decomposition - Engine Accel X (10s segment)')

for j in range(1, n_levels + 1):
    axes[j].plot(t_a, decomp_a[f'D{j}'], linewidth=0.5, color=f'C{j-1}')
    freq_hi = accel_fs / (2**j)
    freq_lo = accel_fs / (2**(j+1))
    axes[j].set_ylabel(f'D{j}\n({freq_lo:.1f}-{freq_hi:.1f}Hz)')

axes[-1].plot(t_a, decomp_a[f'A{n_levels}'], linewidth=0.5, color='gray')
axes[-1].set_ylabel(f'A{n_levels}\n(baseline)')
axes[-1].set_xlabel('Time (s)')

plt.tight_layout()
plt.savefig('figures/modwt_engine_accel_decomposition.png', dpi=150, bbox_inches='tight')
plt.show()

## 4. Enhanced Feature Extraction

In [ ]:
# Window parameters: ~3 seconds, 50% overlap
window_sec = 3.0
step_sec = 1.5

print(f'Window: {window_sec}s = {int(window_sec * gyro_fs)} samples (gyro) / {int(window_sec * accel_fs)} samples (accel)')
print(f'Step: {step_sec}s, 50% overlap')

In [ ]:
# Extract features from engine gyro (calibrated X-axis)
ext_gyro = FeatureExtractor(fs=gyro_fs)
gyro_features = ext_gyro.process_signal(
    gyro_cal_x, window_sec, step_sec,
    include_wavelet=True, wavelet_level=5,
    include_signal_quality=True,
)
print(f'Engine gyro: {len(gyro_features)} windows, {len(gyro_features.columns)} features')

In [ ]:
# Extract features from engine accel (X-axis)
ext_accel = FeatureExtractor(fs=accel_fs)
accel_features = ext_accel.process_signal(
    accel_x, window_sec, step_sec,
    include_wavelet=True, wavelet_level=5,
    include_signal_quality=True,
)
print(f'Engine accel: {len(accel_features)} windows, {len(accel_features.columns)} features')

In [ ]:
# Extract features from PAMAP2 gyro (if available)
pamap2_gyro_features = None
if pamap2_gyro_x is not None:
    ext_pamap2 = FeatureExtractor(fs=pamap2_fs)
    pamap2_gyro_features = ext_pamap2.process_signal(
        pamap2_gyro_x, window_sec, step_sec,
        include_wavelet=True, wavelet_level=5,
        include_signal_quality=True,
    )
    print(f'PAMAP2 gyro: {len(pamap2_gyro_features)} windows, {len(pamap2_gyro_features.columns)} features')

In [ ]:
# Show feature categories
cols = [c for c in gyro_features.columns if c not in ['window_start_sample', 'window_start_sec']]
categories = {
    'Time domain': [c for c in cols if c in ['mean','variance','rms','skewness','kurtosis','zcr','quantile_25','quantile_50','quantile_75']],
    'Frequency': [c for c in cols if c.startswith(('psd_', 'spectral_'))],
    'ADEV': [c for c in cols if c.startswith('adev_')],
    'MODWT wavelet': [c for c in cols if c.startswith('modwt_')],
    'Signal quality': [c for c in cols if c.startswith('sq_')],
}
for cat, feats in categories.items():
    print(f'{cat}: {len(feats)} features')
print(f'Total: {len(cols)} features per window')

## 5. Divergence Analysis Over Time (Belief Divergence)

In [ ]:
# Build reference from first 10% of engine gyro (baseline/healthy period)
n_baseline = int(len(gyro_cal_x) * 0.10)
baseline_signal = gyro_cal_x[:n_baseline]

ref_dists_gyro = build_reference_distributions(baseline_signal, gyro_fs, level=5)
print(f'Reference built from first {n_baseline:,} samples ({n_baseline/gyro_fs/60:.1f} min)')
print(f'Sub-bands: {list(ref_dists_gyro.keys())}')

In [ ]:
# Compute divergence for each window across the full recording
window_samples = int(window_sec * gyro_fs)
step_samples = int(step_sec * gyro_fs)

div_over_time = []
for i in range(0, len(gyro_cal_x) - window_samples + 1, step_samples):
    window = gyro_cal_x[i:i + window_samples]
    div_feats = compute_window_divergence(window, ref_dists_gyro, level=5)
    div_feats['window_start_sec'] = i / gyro_fs
    div_over_time.append(div_feats)

div_df = pd.DataFrame(div_over_time)
print(f'Divergence computed for {len(div_df)} windows over {div_df["window_start_sec"].max()/3600:.1f} hours')

In [ ]:
# Plot divergence over time
fig, axes = plt.subplots(3, 1, figsize=(14, 10), sharex=True)

# Mean KL divergence
hours = div_df['window_start_sec'] / 3600
axes[0].plot(hours, div_df['div_mean_kl'], linewidth=0.8, color='steelblue')
axes[0].set_ylabel('Mean Symmetric KL')
axes[0].set_title('Wavelet Divergence from Baseline Over Time (Engine Gyro CalX)')
axes[0].axhline(y=div_df['div_mean_kl'].median(), color='orange',
                linestyle='--', alpha=0.7, label='Median')
axes[0].legend()

# Per-sub-band KL
for j in range(1, 6):
    col = f'div_d{j}_kl'
    if col in div_df.columns:
        axes[1].plot(hours, div_df[col], linewidth=0.7, label=f'D{j}', alpha=0.8)
if 'div_a_kl' in div_df.columns:
    axes[1].plot(hours, div_df['div_a_kl'], linewidth=0.7, label='A5', alpha=0.8, linestyle='--')
axes[1].set_ylabel('Symmetric KL')
axes[1].set_title('Per-Sub-Band Divergence')
axes[1].legend(ncol=6, fontsize=8)

# JSD (bounded, easier to interpret)
axes[2].plot(hours, div_df['div_mean_jsd'], linewidth=0.8, color='coral')
axes[2].set_ylabel('Mean JSD')
axes[2].set_xlabel('Time (hours)')
axes[2].set_title('Jensen-Shannon Divergence from Baseline')
axes[2].axhline(y=np.log(2), color='gray', linestyle=':', alpha=0.5, label=f'JSD upper bound = ln(2) = {np.log(2):.3f}')
axes[2].legend()

plt.tight_layout()
plt.savefig('figures/divergence_over_time_gyro.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Same for accel
n_baseline_a = int(len(accel_x) * 0.10)
ref_dists_accel = build_reference_distributions(accel_x[:n_baseline_a], accel_fs, level=5)

window_samples_a = int(window_sec * accel_fs)
step_samples_a = int(step_sec * accel_fs)

div_accel = []
for i in range(0, len(accel_x) - window_samples_a + 1, step_samples_a):
    window = accel_x[i:i + window_samples_a]
    div_feats = compute_window_divergence(window, ref_dists_accel, level=5)
    div_feats['window_start_sec'] = i / accel_fs
    div_accel.append(div_feats)

div_accel_df = pd.DataFrame(div_accel)

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(div_accel_df['window_start_sec']/3600, div_accel_df['div_mean_kl'],
        linewidth=0.8, color='steelblue')
ax.set_ylabel('Mean Symmetric KL')
ax.set_xlabel('Time (hours)')
ax.set_title('Wavelet Divergence from Baseline - Engine Accel X')
plt.tight_layout()
plt.savefig('figures/divergence_over_time_accel.png', dpi=150, bbox_inches='tight')
plt.show()

## 6. Cross-Sensor Divergence

In [ ]:
# Compare baseline distributions across sensors
sensors = {'Engine Gyro': ref_dists_gyro, 'Engine Accel': ref_dists_accel}

if pamap2_gyro_x is not None:
    ref_dists_pamap2 = build_reference_distributions(pamap2_gyro_x[:int(len(pamap2_gyro_x)*0.5)], pamap2_fs, level=5)
    sensors['PAMAP2 Gyro'] = ref_dists_pamap2

# Cross-sensor divergence matrix
sensor_names = list(sensors.keys())
n_sensors = len(sensor_names)
cross_matrix = np.zeros((n_sensors, n_sensors))

for i, name_a in enumerate(sensor_names):
    for j, name_b in enumerate(sensor_names):
        if i <= j:
            xdiv = compute_cross_sensor_divergence(sensors[name_a], sensors[name_b])
            val = xdiv.get('xdiv_mean_kl', 0)
            cross_matrix[i, j] = val
            cross_matrix[j, i] = val

fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cross_matrix, cmap='YlOrRd')
ax.set_xticks(range(n_sensors))
ax.set_yticks(range(n_sensors))
ax.set_xticklabels(sensor_names, rotation=45, ha='right')
ax.set_yticklabels(sensor_names)
for i in range(n_sensors):
    for j in range(n_sensors):
        ax.text(j, i, f'{cross_matrix[i,j]:.3f}', ha='center', va='center', fontsize=10)
plt.colorbar(im, label='Mean Symmetric KL Divergence')
ax.set_title('Cross-Sensor Wavelet Divergence')
plt.tight_layout()
plt.savefig('figures/cross_sensor_divergence.png', dpi=150, bbox_inches='tight')
plt.show()

## 7. Feature Stability Analysis

In [ ]:
# Temporal stability within engine gyro (split into 8 time segments)
stability_temporal = assess_feature_stability(gyro_features, n_segments=8)

print('Top 10 most STABLE features (lowest CV):')
print(stability_temporal.head(10)[['feature', 'mean', 'cv', 'stability_rank']].to_string(index=False))
print()
print('Top 10 most SENSITIVE features (highest CV):')
print(stability_temporal.tail(10)[['feature', 'mean', 'cv', 'stability_rank']].to_string(index=False))

In [ ]:
# Visualize stability ranking
fig, ax = plt.subplots(figsize=(14, 6))
colors = ['steelblue' if cv < 0.1 else 'orange' if cv < 0.5 else 'coral'
          for cv in stability_temporal['cv']]
ax.barh(range(len(stability_temporal)), stability_temporal['cv'], color=colors)
ax.set_yticks(range(len(stability_temporal)))
ax.set_yticklabels(stability_temporal['feature'], fontsize=7)
ax.set_xlabel('Coefficient of Variation (CV)')
ax.set_title('Feature Stability Ranking (Engine Gyro) — Lower CV = More Stable')
ax.axvline(x=0.1, color='green', linestyle='--', alpha=0.5, label='Stable threshold (0.1)')
ax.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='Sensitive threshold (0.5)')
ax.legend()
plt.tight_layout()
plt.savefig('figures/feature_stability_ranking.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Cross-condition stability (engine gyro vs engine accel vs PAMAP2)
condition_dfs = {'engine_gyro': gyro_features, 'engine_accel': accel_features}
if pamap2_gyro_features is not None:
    condition_dfs['pamap2_gyro'] = pamap2_gyro_features

stability_cross = assess_cross_condition_stability(condition_dfs)
print('Top 15 features most STABLE across sensors/datasets:')
print(stability_cross.head(15)[['feature', 'cv_across_conditions', 'stability_rank']].to_string(index=False))

## 8. Quality Vector Construction

In [ ]:
# Select quality vector features based on temporal stability
quality_features = select_quality_features(
    stability_temporal,
    n_stable=8,
    n_sensitive=4,
    cv_col='cv',
    mandatory_features=DEFAULT_MANDATORY_FEATURES,
)

print(f'Quality vector composition ({len(quality_features["all"])} features):')
print(f'  Mandatory ({len(quality_features["mandatory"])}): {quality_features["mandatory"]}')
print(f'  Stable    ({len(quality_features["stable"])}): {quality_features["stable"]}')
print(f'  Sensitive ({len(quality_features["sensitive"])}): {quality_features["sensitive"]}')

In [ ]:
# Build quality vector for engine gyro
# Use first 10% as reference for normalization
n_ref_windows = len(gyro_features) // 10
ref_df = gyro_features.iloc[:n_ref_windows]

ref_stats = {}
for feat in quality_features['all']:
    if feat in ref_df.columns:
        ref_stats[feat] = (float(ref_df[feat].mean()), float(ref_df[feat].std()) + 1e-12)

qv_df = build_quality_vector(
    gyro_features, quality_features,
    normalize=True, reference_stats=ref_stats
)

print(f'Quality vector shape: {qv_df.shape}')
print(f'Health score range: [{qv_df["qv_health_score"].min():.3f}, {qv_df["qv_health_score"].max():.3f}]')
if 'qv_degradation_score' in qv_df.columns:
    print(f'Degradation score range: [{qv_df["qv_degradation_score"].min():.3f}, {qv_df["qv_degradation_score"].max():.3f}]')

In [ ]:
# Plot quality vector over time
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

hours_qv = qv_df['window_start_sec'] / 3600

axes[0].plot(hours_qv, qv_df['qv_health_score'], linewidth=0.8, color='steelblue')
axes[0].set_ylabel('Health Score\n(lower = healthier)')
axes[0].set_title('Quality Vector Over Time — Engine Gyro CalX')
# Alert threshold at 95th percentile of baseline period
baseline_health = qv_df['qv_health_score'].iloc[:n_ref_windows]
threshold = baseline_health.quantile(0.95)
axes[0].axhline(y=threshold, color='orange', linestyle='--', alpha=0.7,
                label=f'Alert threshold ({threshold:.2f})')
axes[0].legend()

if 'qv_degradation_score' in qv_df.columns:
    axes[1].plot(hours_qv, qv_df['qv_degradation_score'], linewidth=0.8, color='coral')
    axes[1].set_ylabel('Degradation Score\n(higher = more degradation)')
    axes[1].set_title('Degradation Sensitivity Score')

axes[1].set_xlabel('Time (hours)')
plt.tight_layout()
plt.savefig('figures/quality_vector_timeline.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Build quality vector for accel too
n_ref_a = len(accel_features) // 10
ref_df_a = accel_features.iloc[:n_ref_a]
ref_stats_a = {}
for feat in quality_features['all']:
    if feat in ref_df_a.columns:
        ref_stats_a[feat] = (float(ref_df_a[feat].mean()), float(ref_df_a[feat].std()) + 1e-12)

qv_accel = build_quality_vector(
    accel_features, quality_features,
    normalize=True, reference_stats=ref_stats_a
)

fig, ax = plt.subplots(figsize=(14, 4))
ax.plot(qv_accel['window_start_sec']/3600, qv_accel['qv_health_score'],
        linewidth=0.8, color='steelblue')
ax.set_ylabel('Health Score')
ax.set_xlabel('Time (hours)')
ax.set_title('Quality Vector Health Score - Engine Accel X')
plt.tight_layout()
plt.savefig('figures/quality_vector_accel.png', dpi=150, bbox_inches='tight')
plt.show()

## 9. Validation: Synthetic Degradation Detection

In [ ]:
# Take a clean segment from engine gyro and inject progressive degradation
clean_duration = 60  # seconds
clean_samples = int(clean_duration * gyro_fs)
clean_segment = gyro_cal_x[:clean_samples].copy()

def inject_noise(signal, severity):
    """Add Gaussian noise proportional to severity * signal std."""
    noise = np.random.randn(len(signal)) * severity * np.std(signal)
    return signal + noise

def inject_drift(signal, severity):
    """Add linear drift proportional to severity * signal range."""
    drift = np.linspace(0, severity * np.ptp(signal), len(signal))
    return signal + drift

# Test progressive noise injection
severities = np.linspace(0, 2.0, 11)
health_noise = []
health_drift = []

for sev in severities:
    # Noise test
    degraded = inject_noise(clean_segment, sev)
    feat_df = ext_gyro.process_signal(degraded, window_sec, step_sec,
                                      include_signal_quality=True)
    qv = build_quality_vector(feat_df, quality_features,
                              normalize=True, reference_stats=ref_stats)
    health_noise.append(qv['qv_health_score'].mean())
    
    # Drift test
    degraded_d = inject_drift(clean_segment, sev)
    feat_df_d = ext_gyro.process_signal(degraded_d, window_sec, step_sec,
                                        include_signal_quality=True)
    qv_d = build_quality_vector(feat_df_d, quality_features,
                                normalize=True, reference_stats=ref_stats)
    health_drift.append(qv_d['qv_health_score'].mean())

fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes[0].plot(severities, health_noise, 'o-', color='steelblue', linewidth=2)
axes[0].set_xlabel('Noise Severity')
axes[0].set_ylabel('Mean Health Score')
axes[0].set_title('Response to Noise Injection')
axes[0].grid(True, alpha=0.3)

axes[1].plot(severities, health_drift, 's-', color='coral', linewidth=2)
axes[1].set_xlabel('Drift Severity')
axes[1].set_ylabel('Mean Health Score')
axes[1].set_title('Response to Bias Drift')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Quality Vector Response to Synthetic Degradation', fontweight='bold')
plt.tight_layout()
plt.savefig('figures/degradation_response_validation.png', dpi=150, bbox_inches='tight')
plt.show()

## 10. Summary

In [ ]:
# Summary table of quality vector features
summary_data = []
for feat in quality_features['all']:
    role = []
    if feat in quality_features['mandatory']:
        role.append('Mandatory')
    if feat in quality_features['stable']:
        role.append('Stable')
    if feat in quality_features['sensitive']:
        role.append('Sensitive')
    
    row = stability_temporal[stability_temporal['feature'] == feat]
    cv_val = row['cv'].values[0] if len(row) > 0 else np.nan
    mean_val = row['mean'].values[0] if len(row) > 0 else np.nan
    
    summary_data.append({
        'Feature': feat,
        'Role': ' + '.join(role),
        'CV': f'{cv_val:.4f}' if not np.isnan(cv_val) else 'N/A',
        'Mean': f'{mean_val:.4f}' if not np.isnan(mean_val) else 'N/A',
    })

summary_df = pd.DataFrame(summary_data)
print('Quality Vector Feature Summary:')
print(summary_df.to_string(index=False))
print(f'\nTotal quality vector size: {len(quality_features["all"])} features + 2 composite scores')

In [ ]:
# Export quality vector for prediction team
print('Quality vector columns available for prediction:')
print(qv_df.columns.tolist())
print(f'\nShape: {qv_df.shape} (windows x features)')
print(f'\nFirst 5 rows:')
qv_df.head()